# Creación de paquetes ARP

### Descripción: 
Con este código se va a intentar crear diferentes tipos de paquetes ARP, mediante IA generativa. Se pretende pasar una descripción de estos paquetes al modelo y que este sea capaz de crear todos los campos necesarios para replicar un paquete con esas mismas especificaciones.

In [ ]:
import random
import re
import pickle
import os 
import openai
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = 'FILL'
client = OpenAI()

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether,ARP

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de un paquete ICMP

<center><img src="https://dis.um.es/~lopezquesada/documentos/IES_1314/LMSGI/curso/xhtml/xhtml11/imagenes/arp.jpg">
    
<center> Esta imagen es genérica para una conversación ARP.


In [2]:
system_message = "In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.\n\n"
system_message += "ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast\n"

system_message += "This is an ARP specific IP query summary:\n"
system_message += '75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1\n'
system_message += "This is the command to generate this packet\n"
system_message += 'scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:0d:09:e2:5d")\n\n'

system_message += "This is an ARP broadcast query summary:\n"
system_message += 'c2:3d:19:6c:00:01 Broadcast ARP 64 Who has 192.168.121.2? Tell 192.168.121.253\n'
system_message += "This is the command to generate this packet\n"
system_message += 'scapy.Ether(src="c2:3d:19:6c:00:01", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=1, pdst="192.168.121.2", psrc="192.168.121.253", hwdst="00:00:00:00:00:00", hwsrc="c2:3d:19:6c:00:01")\n\n'

system_message += "This is an ARP response summary:\n"
system_message += "c2:3c:19:6c:00:01 c2:3d:19:6c:00:01 ARP 60 10.0.0.2 is at c2:3c:19:6c:00:01"
system_message += "This is the command to generate the packet:\n"
system_message += 'scapy.Ether(src="c2:3c:19:6c:00:01", dst="c2:3d:19:6c:00:01")/scapy.ARP(op=2, psrc="10.0.0.2", hwsrc="c2:3c:19:6c:00:01", hwdst="c2:3d:19:6c:00:01",  pdst=socket.inet_ntoa(struct.pack(">I", random.randint(1, 0xffffffff))))\n\n'

system_message += "This is an Gratuitous ARP summary:\n"
system_message += "c2:3c:19:6c:00:01     Broadcast             ARP      60     Gratuitous ARP for 10.0.0.2"
system_message += "This is the command to generate the packet:\n"
system_message += 'scapy.Ether(src="c2:3c:19:6c:00:01", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=2, psrc="10.0.0.2", hwsrc="c2:3c:19:6c:00:01", hwdst="FF:FF:FF:FF:FF:FF",  pdst="10.0.0.2")\n\n'

system_message += 'Take your time (few seconds) to think if the packets and their variables make sense for an ARP packet, if not, correct them\n'

In [3]:
print(system_message)

In the next lines, I'll provide you with some packet descriptions for different types of ARP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

ARP packets can be 2 types, queries and responses. Inside queries, they can have as dst a specific IP or to broadcast
This is an ARP specific IP query summary:
75:3a:0d:09:e2:5d 39:a6:a8:a5:73:c1 ARP 42 Who has 192.168.2.102? Tell 192.168.2.1
This is the command to generate this packet
scapy.Ether(src="75:3a:0d:09:e2:5d", dst="39:a6:a8:a5:73:c1")/scapy.ARP(op=1, pdst="192.168.2.102", psrc="192.168.2.1", hwdst="00:00:00:00:00:00", hwsrc="75:3a:0d:09:e2:5d")

This is an ARP broadcast query summary:
c2:3d:19:6c:00:01 Broadcast ARP 64 Who has 192.168.121.2? Tell 192.168.121.253
This is the command 

## Carga de posibles descripciones

In [4]:
with open("../../data/text/ARP_TheUltimatePCAP_v2.txt","r") as f:
    packets_summary = f.read().splitlines()
len(packets_summary)

999

In [5]:
test_group = random.choices(packets_summary,k=200)

### Carga de posibles conversaciones anteriores para concatenarlas

In [6]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [7]:
try:
    responses = pickle.load( open("../../data/Packets/ARP/pickle/ARP_packet_pairs_Aday_v3.pkl", "rb" ))
except:
    responses = []

## Generación de comandos de paquetes conversaciones TCP

In [8]:
index = 0

In [9]:
for i in range(int(len(test_group)/10)):
    
    summaries = test_group[index:index+10]

    index += 10
    #print(index)
    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter " +str(i+1) +")")
    

    text_sum = ""
    
    query_content = "\nBelow, I'll give you the descriptions of the packets you have to generate, the previous where just examples\n\n"
    
    for summary in summaries:
        query_content += "This is a packet summary:\n"
        query_content += summary + "\n"
        text_sum += summary + "\n"
        
    query_content += "\nGenerate python code for creating JUST the packets of the summaries given with scapy framework and put them all in a list named pkt_list. DO NOT create commands for other types of ARP. Dont give back any comments or imports, just bare code and commands.\n"
    query_content += "Use the following structure for the pkt_list: pkt_list[scapy.Ether(...)/scapy.ARP(..), scapy.Ether(...)/scapy.ARP(..), ...]"
    
    print(query_content)
    
    messages=[{"role": "system", "content": system_message},
        {"role": "user", "content": query_content}]
    
    #Esta es la petición que le mandas al modelo de IA
    completion = client.chat.completions.create(
        model="gpt-4o", 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)

    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    codigo = responses[-1].completion.choices[0].message.content
    
    if codigo.startswith("```python"):
        codigo = codigo[len("```python"):].lstrip()  # Elimina ```python al inicio
    if codigo.endswith("```"):
        codigo = codigo[:-len("```")].rstrip()  # Elimina ``` al final
    
    exec(codigo)
    
    with open("../../data/Packets/ARP/pcap/ARPgenerated_gpt_4o_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_list, append=True)
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Packets/ARP/pickle/ARP_packet_pairs_Aday_v3.pkl", "wb" ) )
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 1)

Below, I'll give you the descriptions of the packets you have to generate, the previous where just examples

This is a packet summary:
33:44:55:66:77:88     33:44:55:66:77:89     ARP      60     Who has 172.20:30.2? Tell 172.20.30.1
This is a packet summary:
12:34:56:78:9a:bc     12:34:56:78:9a:bd     ARP      60     Who has 10.200.200.2? Tell 10.200.200.1
This is a packet summary:
01:23:45:67:89:ab     01:23:45:67:89:ac     ARP      60     Who has 192.168.100.2? Tell 192.168.100.1
This is a packet summary:
0f:10:11:12:13:15     0f:10:11:12:13:14     ARP      60     Who has 172.16.20.1? Tell 172.16.20.2
This is a packet summary:
10:20:30:40:50:62     10:20:30:40:50:61     ARP      60     Who has 172.16.5.1? Tell 172.16.5.2
This is a packet summary:
01:ab:cd:ef:23:46     01:ab:cd:ef:23:45     ARP      60     172.20.10.2 is at 01:ab:cd:ef:23:46
This is a packet summary:
55:55:55:55:

...Generated!
Number of completitions done: 5
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 6)

Below, I'll give you the descriptions of the packets you have to generate, the previous where just examples

This is a packet summary:
aa:cc:ee:11:22:33     aa:cc:ee:11:22:34     ARP      60     Who has 10.50.50.2? Tell 10.50.50.1
This is a packet summary:
12:34:56:78:9a:bd     12:34:56:78:9a:bc     ARP      60     10.200.200.2 is at 12:34:56:78:9a:bd
This is a packet summary:
10:20:30:40:50:62     10:20:30:40:50:61     ARP      60     172.16.5.2 is at 10:20:30:40:50:62
This is a packet summary:
e4:8d:8c:7b:00:03     Broadcast             ARP      60     Gratuitous ARP for 172.16.0.3 (Request)
This is a packet summary:
aa:aa:aa:aa:aa:02     aa:aa:aa:aa:aa:01     ARP      60     Who has 10.1.1.1? Tell 10.1.1.2
This is a packet summary:
bb:cc:dd:ee:ff:02     bb:cc:dd:ee:ff:01     ARP      60     192.168.1.11 is at bb:cc:dd:ee:ff:02
This 

...Generated!
Number of completitions done: 10
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 11)

Below, I'll give you the descriptions of the packets you have to generate, the previous where just examples

This is a packet summary:
88:88:88:88:88:02     88:88:88:88:88:01     ARP      60     10.50.50.2 is at 88:88:88:88:88:02
This is a packet summary:
aa:bb:cc:dd:ee:04     aa:bb:cc:dd:ee:03     ARP      60     Who has 172.16.100.1? Tell 172.16.100.2
This is a packet summary:
10:20:30:40:50:62     10:20:30:40:50:61     ARP      60     172.16.5.2 is at 10:20:30:40:50:62
This is a packet summary:
bb:bb:bb:bb:bb:01     Broadcast             ARP      60     Gratuitous ARP for 192.168.1.1 (Reply)
This is a packet summary:
aa:cc:ee:11:22:33     aa:cc:ee:11:22:34     ARP      60     Who has 10.50.50.2? Tell 10.50.50.1
This is a packet summary:
11:22:33:44:55:67     11:22:33:44:55:66     ARP      60     Who has 10.10.10.1? Tell 10.10:10.2

...Generated!
Number of completitions done: 15
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 16)

Below, I'll give you the descriptions of the packets you have to generate, the previous where just examples

This is a packet summary:
08:09:0a:0b:0c:0e     08:09:0a:0b:0c:0d     ARP      60     Who has 192.168.53.1? Tell 192.168.53.2
This is a packet summary:
08:09:0a:0b:0c:0d     Broadcast             ARP      60     Gratuitous ARP for 192.168.53.1 (Request)
This is a packet summary:
01:ab:cd:ef:23:45     Broadcast             ARP      60     Gratuitous ARP for 172.20.10.1 (Request)
This is a packet summary:
77:88:99:aa:bb:cd     77:88:99:aa:bb:cc     ARP      60     10.20.20.2 is at 77:88:99:aa:bb:cd
This is a packet summary:
bb:cc:dd:ee:ff:01     bb:cc:dd:ee:ff:02     ARP      60     Who has 192.168.1.11? Tell 192.168.1.10
This is a packet summary:
12:34:56:78:9a:bd     12:34:56:78:9a:bc     ARP      60     172.16.1.2 is at 12:34

...Generated!
Number of completitions done: 20


In [70]:
exec(responses[-1].completion.choices[0].message.content[10:-4])
    
with open("../../data/Packets/ARP/pcap/ARPgenerated_gpt_4o_conv.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)

# Guardamos tanto la petición como la respuesta en un archivo
pickle.dump(responses, open( "../../data/Packets/ARP/pickle/ARP_packet_pairs_Aday.pkl", "wb" ) )

print("Number of completitions done: " + str(len(responses)))

Number of completitions done: 8


In [217]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

a4:9b:4f:53:00:0d     a4:9b:4f:53:00:0e     ARP      60     192.168.2.13 is at a4:9b:4f:53:00:0d
a4:9b:4f:53:00:0e     Broadcast             ARP      60     Gratuitous ARP for 192.168.2.14 (Reply)



pkt_list = [
    scapy.Ether(src="a4:9b:4f:53:00:0d", dst="a4:9b:4f:53:00:0e")/scapy.ARP(op=2, psrc="192.168.2.13", hwsrc="a4:9b:4f:53:00:0d", hwdst="a4:9b:4f:53:00:0e"),
    scapy.Ether(src="a4:9b:4f:53:00:0e", dst="FF:FF:FF:FF:FF:FF")/scapy.ARP(op=2, psrc="192.168.2.14", hwsrc="a4:9b:4f:53:00:0e", hwdst="FF:FF:FF:FF:FF:FF", pdst="192.168.2.14")
]


### Generación de paquetes

In [220]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<Ether  dst=a4:9b:4f:53:00:0e src=a4:9b:4f:53:00:0d type=ARP |<ARP  op=is-at hwsrc=a4:9b:4f:53:00:0d psrc=192.168.2.13 hwdst=a4:9b:4f:53:00:0e |>>, <Ether  dst=FF:FF:FF:FF:FF:FF src=a4:9b:4f:53:00:0e type=ARP |<ARP  op=is-at hwsrc=a4:9b:4f:53:00:0e psrc=192.168.2.14 hwdst=FF:FF:FF:FF:FF:FF pdst=192.168.2.14 |>>]


In [222]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 2 packets.


### Introducción en un pcap

In [225]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Packets/ARP/pcap/Validation_v2_ARPgenerated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)